In [17]:
import pandas as pd
import plotly.express as px
import numpy as np


df_green = pd.read_csv('datasets/iso_green.csv')
\
def filter_outliers(data, threshold=2):
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    filtered_data = data[z_scores < threshold]
    return filtered_data

filtered_technologies = filter_outliers(df_green['Development of environment-related technologies, % inventions worldwide'])
filtered_emissions = filter_outliers(df_green['Demand-based CO2 emissions'])
filtered_years = df_green.loc[filtered_technologies.index, 'Year']


filtered_df = pd.DataFrame({'Year': filtered_years, 'Technologies': filtered_technologies, 'Emissions': filtered_emissions})


filtered_df = filtered_df.dropna()

avg_technologies = filtered_df.groupby('Year')['Technologies'].mean().values
avg_emissions = filtered_df.groupby('Year')['Emissions'].mean().values
filtered_years = filtered_df['Year'].unique()

averages_df = pd.DataFrame({'Year': filtered_years, 'Avg Technologies': avg_technologies, 'Avg Emissions': avg_emissions})

fig3d = px.scatter_3d(averages_df, x='Year', y='Avg Technologies', z='Avg Emissions')

fig3d.update_traces(
    marker=dict(
        size=7,  
    )
)

fig3d.update_layout(
    height=600, width=1200,
    scene=dict(
        xaxis_title='Year',
        yaxis_title='AVG Dev environment related technologies',
        zaxis_title='Average Carbon emissions',
        
        camera=dict(
            eye=dict(x=1.8, y=-1.8, z=0.6)  
        )
    ),
    title='Average development of environmental technologies and carbon emissions per year'
)

fig3d.add_annotation(
    text='Fig 8: This graph shows the Average Development of environment related technologies as a percentage of worldwide inventions per year',
    xref="paper",
    yref="paper",
    x=0.5,
    y=1,
    showarrow=False,
    font=dict(size=12)
)
fig3d.add_annotation(
    text='compared to the year and the average amount of carbon emissions for that year',
    xref="paper",
    yref="paper",
    x=0.5,
    y=0.95,
    showarrow=False,
    font=dict(size=12)
)

%store fig3d

fig3d.show()


Stored 'fig3d' (Figure)


In [20]:
import plotly.graph_objects as go
import statsmodels.api as sm

df_green = pd.read_csv('datasets/iso_green.csv')

def filter_outliers(data, threshold=2):
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    filtered_data = data[z_scores < threshold]
    return filtered_data

filtered_technologies = filter_outliers(df_green['Development of environment-related technologies, % inventions worldwide'])
filtered_emissions = filter_outliers(df_green['Demand-based CO2 emissions'])
filtered_years = df_green.loc[filtered_technologies.index, 'Year']

filtered_df = pd.DataFrame({'Year': filtered_years, 'Technologies': filtered_technologies, 'Emissions': filtered_emissions})

filtered_df = filtered_df.dropna()

avg_technologies = filtered_df.groupby('Year')['Technologies'].mean().values
avg_emissions = filtered_df.groupby('Year')['Emissions'].mean().values
filtered_years = filtered_df['Year'].unique()

averages_df = pd.DataFrame({'Year': filtered_years, 'Avg Technologies': avg_technologies, 'Avg Emissions': avg_emissions})

X = sm.add_constant(avg_technologies)  
model = sm.OLS(avg_emissions, X)
results = model.fit()
reg_line = results.predict(X)

fig3dreg = go.Figure()

for year, tech, emiss in zip(filtered_years, avg_technologies, avg_emissions):
    fig3dreg.add_annotation(
        x=tech,
        y=emiss,
        text=str(int(year)),  
        showarrow=False,
        font=dict(size=12),
        xshift=5,
        yshift=-10
    )

fig3dreg.add_trace(go.Scatter(
    x=avg_technologies,
    y=avg_emissions,
    mode='markers',
    marker=dict(color='rgb(31, 119, 180)'),
    name='Data per year'
))

fig3dreg.add_trace(go.Scatter(
    x=avg_technologies,
    y=reg_line,
    mode='lines',
    line=dict(color='red'),
    name='Regression Line'
))

fig3dreg.update_layout(
    xaxis_title='Average Development (environment related) technologies',
    yaxis_title='Average Carbon emissions',
    title='Regression line of the Average carbon emissions and development in environmental technologies per year',
    height=600, width=1200
)
fig3dreg.add_annotation(
    text='Fig 9: Regressional line of the Average Carbon emission data per year compared to the Average Development in environmental related technologies per year',
    xref="paper",
    yref="paper",
    x=0.5,
    y=-0.15,
    showarrow=False,
    font=dict(size=12)
)

%store fig3dreg
fig3dreg.show()


Stored 'fig3dreg' (Figure)
